In [4]:
import polars as pl
import os
from tqdm.notebook import tqdm

base_path = "RFSD"
dfs = []

good_corr_lines = ['line_1230',
 'line_1250',
 'line_1500',
 'line_1520',
 'line_2110',
 'line_2120',
 'line_2200',
 'line_2300',
 'line_2350',
 'line_2400',
 'line_2500']

needed_initial_cols = ['inn', 'year', 'region', 'region_taxcode', 'age', 'dissolution_date', 'filed', 'okved', 'okved_section', 'okopf', 'okogu', 'okfc', 'lon', 'lat', 'geocoding_quality'] + good_corr_lines

with pl.StringCache():
    for folder_name in tqdm(os.listdir(base_path)):
        if folder_name.startswith("year="):
            year = int(folder_name.split("=")[1])
            # if year == 2023:
            folder_path = os.path.join(base_path, folder_name)
            df = pl.read_parquet(os.path.join(folder_path, "*.parquet"))
            df = df.with_columns(pl.lit(year).alias("year"))
            df = df.filter(pl.col('inn').is_not_null() & pl.col('region').is_not_null() & (pl.col('outlier') == 0))
            minified = df.select(needed_initial_cols)

            del df
            
            dfs.append(minified.to_pandas())

            del minified
            
    # RFSD = pd.DataFrame.concat(dfs)

  0%|          | 0/13 [00:00<?, ?it/s]

In [5]:
import pandas as pd

In [6]:
main_df = pd.concat(dfs)

/tmp/ipykernel_2126/111979488.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  main_df = pd.concat(dfs)


In [7]:
len(main_df)

56413605

In [8]:
sorted_by_year = main_df.sort_values(by='year')

In [11]:
import json
combined_workers_count = json.load(open('combined_workers_count.json', 'r'))

workers_count_by_comp = [combined_workers_count.get(str(i), None) for i in sorted_by_year.inn]

In [12]:
sorted_by_year['workers_count'] = workers_count_by_comp

In [13]:
actual = sorted_by_year[(sorted_by_year.filed == 1) | (sorted_by_year.workers_count.isnull() == False)].drop_duplicates('inn', keep='last')

In [14]:
actual = actual.reset_index(drop=True)


In [15]:
len(actual)

5038803

In [16]:
actual.head()

,inn,year,region,region_taxcode,age,dissolution_date,filed,okved,okved_section,okopf,...,line_1500,line_1520,line_2110,line_2120,line_2200,line_2300,line_2350,line_2400,line_2500,workers_count
0,7842356811,2011,sankt-petersburg,78,4.0,2011-09-14,0.0,47.91.20,G,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,7838455351,2011,moscow city,78,0.0,NaT,0.0,64.99.10,K,12300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,7805196817,2011,sankt-petersburg,78,10.0,2011-02-03,0.0,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,7731627713,2011,moscow city,77,2.0,NaT,0.0,46.72.23,G,12300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,7727682302,2011,moscow city,77,2.0,2011-03-17,0.0,58,J,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [17]:
actual.to_csv('rfsd_data_with_workers_and_historical_data.csv')

In [18]:
actual = actual[actual.dissolution_date.isnull()]

In [19]:
len(actual)

2656150

In [20]:
len(actual[actual.workers_count > 0])

1777874

In [21]:
actual.workers_count.sum()

np.float64(19020429.0)

In [23]:
needed_test_cols = ['inn', 'region', 'region_taxcode', 'age', 'okved', 'okved_section', 'okopf', 'okogu', 'okfc'] + good_corr_lines

In [24]:
actual_to_predict = actual[actual.workers_count.isnull() & ~actual.okved.isnull() & ~actual.okopf.isnull()][needed_test_cols]


In [25]:
import catboost

In [28]:
model = catboost.CatBoostRegressor()
model.load_model('workers_count_prediction_model.cbm')

In [29]:
actual_to_predict['predicted_workers_count'] = model.predict(actual_to_predict)

In [60]:
actual.loc[actual_to_predict.index, 'predicted_workers_count'] = actual_to_predict['predicted_workers_count']

In [33]:
actual.workers_count.sum(), actual.predicted_workers_count.sum()

(np.float64(19020429.0), np.float64(11251148.203448808))

In [34]:
actual.workers_count.sum()+ actual.predicted_workers_count.sum()

np.float64(30271577.20344881)

In [50]:
import numpy as np

In [69]:
actual['predicted_workers_count'] = actual.predicted_workers_count.clip(lower=0)

In [70]:
actual['predicted_workers_count']

1                NaN
3                NaN
275         4.306216
309        11.970730
357        13.197641
             ...    
5038798    18.239452
5038799          NaN
5038800          NaN
5038801          NaN
5038802          NaN
Name: predicted_workers_count, Length: 2656150, dtype: float64

In [72]:
actual['combined_workers_count'] = actual.workers_count.fillna(0) + actual.predicted_workers_count.fillna(0)

In [65]:
# actual = pd.read_pickle('rfsd_data_with_workers_and_historical_data_and_predicted_workers_count.pck')

In [74]:
actual.to_pickle('rfsd_data_with_workers_and_historical_data_and_predicted_workers_count.pck')

In [40]:
actual.dtypes

inn                                object
year                                int32
region                             object
region_taxcode                   category
age                               float64
dissolution_date           datetime64[ms]
filed                             float64
okved                              object
okved_section                    category
okopf                              object
okogu                              object
okfc                               object
lon                                object
lat                                object
geocoding_quality                category
line_1230                         float64
line_1250                         float64
line_1500                         float64
line_1520                         float64
line_2110                         float64
line_2120                         float64
line_2200                         float64
line_2300                         float64
line_2350                         

In [43]:
actual.workers_count.sum()

np.float64(19020429.0)

In [71]:
actual.predicted_workers_count.sum()

np.float64(11516961.226479687)

In [67]:
actual.combined_workers_count.sum()

np.float64(30271577.203448817)

In [73]:
actual.combined_workers_count.sum()

np.float64(30537390.2264797)